In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())

In [ ]:
%cd /content/gdrive/MyDrive/chest_xray

In [ ]:

import numpy as np
import pandas as pd

import os
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization, Input, MaxPooling2D
from keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix


EPOCHS = 20
INIT_LR = 0.000001
IMG_SIZE = 224
TITLES = ['PNEUMONIA', 'NORMAL']
PATH = ''

In [ ]:
def get_data(data_dir):
    data = []
    
    for title in TITLES:
        print(f"Loading {title} ...")
        path = os.path.join(data_dir, title)
        class_number = TITLES.index(title)
        for img_file in os.scandir(path):
            if img_file.name == ".DS_Store" or not img_file.is_file():
                continue
            try:
                img_arr = cv2.imread(img_file.path, cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                data.append([resized_arr, class_number])
            except Exception as e:
                print(f"Error loading {img_file.name}: {e}")    
    print(f"Finished loading {len(data)} images from {data_dir}.")
    
    return np.array(data)

In [ ]:
train_data = get_data('train')
test_data = get_data('test')
val_data = get_data('val')

In [ ]:
import matplotlib.pyplot as plt

target_labels = ["Pneumonia", "Normal"]
t = [target_labels[i[1]] for i in train_data]

counts = [t.count(label) for label in target_labels]

plt.pie(counts, labels=target_labels, autopct='%1.1f%%', textprops={'fontsize': 24})
plt.title("Distribution of Target Labels", fontsize=26)
plt.show()


In [ ]:
target_labels = ["Pneumonia", "Normal"]
t = [target_labels[i[1]] for i in train_data]

my_palette = sns.color_palette(['#9b59b6', '#3498db'])
sns.set_palette(my_palette)

sns.countplot(x=t)
plt.xlabel('Class', fontsize=22)
plt.ylabel('Count', fontsize=22)
plt.title('Distribution of Classes', fontsize=26)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(train_data[0][0], cmap='gray')
plt.title(TITLES[train_data[0][1]], fontsize=22)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

# Display last image in train_data with title
plt.figure(figsize=(5, 5))  # Set plot size
plt.imshow(train_data[-1][0], cmap='gray')  # Show image data
plt.title(TITLES[train_data[-1][1]], fontsize=22)  # Set title based on class label
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
X_train, y_train = zip(*[(feature, title) for feature, title in train_data])
X_val, y_val = zip(*[(feature, title) for feature, title in val_data])
X_test, y_test = zip(*[(feature, title) for feature, title in test_data])

In [ ]:
X_train = np.array(X_train) / 255
X_val = np.array(X_val) / 255
X_test = np.array(X_test) / 255

In [ ]:
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array(y_train)

X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = np.array(y_val)

X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array(y_test)

In [ ]:
data_mod = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=20, 
        zoom_range=0.2,
        width_shift_range=0.2, 
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')
        
data_mod.fit(X_train)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def convolutional_block(x, filters, kernel_size, strides, dropout_rate):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=2, padding='same')(x)
    return x

def fully_connected_block(x, units, dropout_rate):
    x = Dense(units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    return x

def create_model(input_shape=(224, 224, 1)):
    inputs = Input(shape=input_shape)
    x = convolutional_block(inputs, filters=32, kernel_size=(3, 3), strides=1, dropout_rate=0.0)
    x = convolutional_block(x, filters=64, kernel_size=(3, 3), strides=1, dropout_rate=0.1)
    x = convolutional_block(x, filters=64, kernel_size=(3, 3), strides=1, dropout_rate=0.0)
    x = convolutional_block(x, filters=128, kernel_size=(3, 3), strides=1, dropout_rate=0.2)
    x = convolutional_block(x, filters=256, kernel_size=(3, 3), strides=1, dropout_rate=0.2)
    x = Flatten()(x)
    x = fully_connected_block(x, units=128, dropout_rate=0.2)
    outputs = Dense(units=1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

def compile_model(model):
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

def print_summary(model):
    model.summary()

In [ ]:
model = create_model()
compile_model(model)
print_summary(model)

In [ ]:
lrr = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=INIT_LR)

chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)

history = model.fit(data_mod.flow(X_train,y_train, batch_size = 32),
                    epochs = 20,
                    validation_data = data_mod.flow(X_val, y_val),
                    callbacks = [lrr, chkpt])



In [ ]:
model.load_weights('best_model_todate')

In [ ]:
print("Loss of the model is - " , model.evaluate(X_test, y_test)[0])
print("Accuracy of the model is - " , model.evaluate(X_test, y_test)[1]*100 , "%")

In [ ]:
fig, ax = plt.subplots(1, 2)
train_acc = history.history['accuracy']
epochs = range(1, len(train_acc) + 1)
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

fig.set_size_inches(20, 10)

ax[0].plot(epochs, train_acc, 'go-', label='Training Accuracy')
ax[0].plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs, train_loss, 'g-o', label='Training Loss')
ax[1].plot(epochs, val_loss, 'r-o', label='Validation Loss')
ax[1].set_title('Training & Validation Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
from keras.models import load_model
model = load_model('model20Epoch91AccLRRCHKPTES.h5')

In [ ]:
predictions = (model.predict(X_test) > 0.5).astype("int32")
TITLES = ["Pneumonia", "Normal"]
conf_matrix = confusion_matrix(y_test, predictions)
conf_df = pd.DataFrame(conf_matrix, index=TITLES, columns=TITLES)

In [ ]:
conf_df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(conf_df, cmap="YlOrBr", linecolor='black', linewidth=1, 
            annot=True, fmt='g', cbar=False,
            square=True, xticklabels=TITLES, yticklabels=TITLES, annot_kws={"fontsize": 22},
            ax=ax)
ax.set_xlabel('Predicted Class', fontsize=20)
ax.set_ylabel('True Class', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=20)
plt.title("Confusion Matrix", fontsize=24)
plt.show()

In [ ]:
model.save("model20Epoch91AccLRRCHKPTES.h5")

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)